In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd

# Always make it pretty.
plt.style.use('ggplot')

In [7]:
df1 = pd.read_csv("data/accidents_2005_to_2007.csv")
df2 = pd.read_csv("data/accidents_2009_to_2011.csv")
df3 = pd.read_csv("data/accidents_2012_to_2014.csv")
traffic = pd.read_csv("data/ukTrafficAADF.csv")

df12 = pd.concat([df1,df2])
df = pd.concat([df12,df3])

df = df.drop_duplicates(subset=['Accident_Index','Date','LSOA_of_Accident_Location','Time','Longitude','Latitude'], keep='first')

df = df[df.Speed_limit !=15]
df = df[(df.Weather_Conditions == 'Fine without high winds') | (df.Weather_Conditions =='Raining without high winds')]

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
del df['Police_Force']
del df['Local_Authority_(Highway)']
del df['Local_Authority_(District)']
del df['Pedestrian_Crossing-Human_Control']
del df['Pedestrian_Crossing-Physical_Facilities']
del df['Did_Police_Officer_Attend_Scene_of_Accident']

In [9]:
severity_totals = df.groupby('Accident_Severity').size().values
#total is 1,469,963 number of accidents 1469963
severity_mus = severity_totals / severity_totals.sum()
severity_mus

array([0.01306109, 0.1370329 , 0.84990601])

In [10]:
#actuals number of accidents of severity 0,1,2, (columns) and Weather conditions sunny (0) and rainy (1) rows
# actuals_severity_weather = df.groupby(['Accident_Severity','Weather_Conditions']).count()['Accident_Index']

actuals_severity_weather = np.transpose(np.reshape(df.groupby(['Accident_Severity','Weather_Conditions']).count()['Accident_Index'].values,(3,2)))
actuals_severity_weather

array([[ 15811, 164200, 996631],
       [  1816,  20737, 150386]])

In [11]:
sums2 = df.groupby(['Accident_Severity','Weather_Conditions']).count().groupby('Accident_Severity').sum()['Accident_Index']
severity_sums = df.groupby(['Accident_Severity','Weather_Conditions']).count().groupby('Accident_Severity').sum()['Accident_Index'].values
severity_mus = severity_sums/severity_sums.sum()
severity_mus

array([0.01306109, 0.1370329 , 0.84990601])

In [12]:
sums = df.groupby(['Weather_Conditions']).count()['Accident_Index']
weather_sums = df.groupby(['Weather_Conditions']).count()['Accident_Index'].values
weather_sums

array([1176642,  172939])

In [18]:
expected_by_weather = np.zeros((2,3))
for i in np.arange(2):
    for j in np.arange(3):
        expected_by_weather[i][j] = weather_sums[i] * severity_mus[j]
expected_by_weather

array([[  15368.22801595,  161238.66707815, 1000035.10490589],
       [   2258.77198405,   23698.33292185,  146981.89509411]])

In [20]:
expected_by_weather = expected_by_weather.round().astype(int)
expected_by_weather
expected_by_weather.sum(axis=1)

array([1176642,  172939])

In [22]:
pvalues = np.zeros(2)

# stats.chisquare(actuals_severity_slimits[0],expected_by_slimit[0])[1]

for i in np.arange(2):
    pvalues[i] = stats.chisquare(actuals_severity_weather[i],expected_by_weather[i])[1]

pvalues

array([8.00584456e-018, 4.77685165e-117])